# 1. Data Analysis of the Influencers:
# 2. Sentiment Analysis of the Influencers: 


In [ ]:
#  TM -Import the Dependencies
import tweepy
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime, timezone
import seaborn as sns
from os import path, makedirs # fetch path and makedirs function from os file
import csv # fetch csv file
from glob import glob # fetching glob function only from the glob lib

In [ ]:
#  TM -Import keys from the config file
# from config import consumer_key, consumer_secret, access_token, access_token_secret

In [ ]:
#  TM - Twitter API Keys
# consumer_key = 'Your Key'
# consumer_secret = 'Your Key'
# access_token = 'Your Key'
# access_token_secret = 'Your Key'

In [ ]:
# Twitter TheDataDash1 API Keys
consumer_key = 'HHEIT7aJjF5SjlhQQ7w40rcZb'
consumer_secret = 'RK0DJHrmwBdSEEs096gQoQBmElEBrtVeRkffveiDLq6RiesUar'
access_token = '1015366295838253057-NXFWxcz1HcCog8CkLhRgN2JV44PQBP'
access_token_secret = 'qMAM1dplnr3vIOWlw2D0ox39R2cgn2S4VPybZadg1bfxN'

In [ ]:
# TM - Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# TM - Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [ ]:
#  TM - Load and read csv file containing the details of the Influencers 
influencer_data_load = "RawData/SentimentInfluencerInputData.csv"
influencer_data_read = pd.read_csv(influencer_data_load)

In [ ]:
#  Team version merged.
# TM - for index, row in Influencers_DF.iterrows():
Data_Influencers_DF = influencer_data_read
Not_Found = 0

#  TM - Define variable for holding tweets for influencer

all_tweet_listing = []

print("-----------Start extraction!!!-----------")

for index, row in Data_Influencers_DF.iterrows():
    target_user = row["Twitter_Handle"]
    Genre =  row["Genre"]
    
    try:
        
        # TM -  retrieve top 200 tweets for influencer
        
        public_tweets = api.user_timeline(target_user, count=200, result_type="recent")

        # TM -  write to all_tweet_listing

        for tweet in public_tweets:
            
            Date = datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S %z %Y').strftime('%m/%d/%Y')

            all_tweet_listing.append({"Influencer":target_user,
                               "Date": Date,
                               "Genre":Genre,
                               "Tweet":tweet["text"],
                               "Retweet_Count":tweet["retweet_count"]})
    
        user_account = api.get_user(target_user)
        user_geo_enabled = user_account["geo_enabled"]
        if (user_geo_enabled == True):
            Data_Influencers_DF.at[index, "Loc"] = user_account["location"]
        else:
            Data_Influencers_DF.at[index, "Loc"] = 'NA'

        if (user_account["lang"] == 'en'):
            Data_Influencers_DF.at[index, "Lang"] = 'Eng'
        else:
            Data_Influencers_DF.at[index, "Lang"] = 'NA'
        
        Data_Influencers_DF.at[index, "Created On"] = datetime.strptime(user_account['created_at'],'%a %b %d %H:%M:%S %z %Y').strftime('%m/%d/%Y')
        
        Data_Influencers_DF.at[index, "Age Of Account"] = (datetime.now(timezone.utc) - datetime.strptime(user_account['created_at'],'%a %b %d %H:%M:%S %z %Y')).days
        
        
       #  TM -  Data_Influencers_DF.at[index, "Real Name"] = user_real_name
        Data_Influencers_DF.at[index, "Tweets"] = user_account["statuses_count"]
        Data_Influencers_DF.at[index, "Followers"] = user_account["followers_count"]
        Data_Influencers_DF.at[index, "Following"] = user_account["friends_count"]
        Data_Influencers_DF.at[index, "Favorites Count"] = user_account["favourites_count"]
        
              
    
    except tweepy.TweepError as e:
        Not_Found = Not_Found + 1
        print(f"exception for {row['Twitter_Handle']}: {e}")

print("----------- Extraction Complete !!!-----------")        
print(Not_Found)

In [ ]:
# TM - translate dict to a dataframe
tweet_listing_pd = pd.DataFrame.from_dict(all_tweet_listing)

In [ ]:
# TM - stats for tweet listing for influencers. PLEASE DON'T REMOVE. Required to quantify digital footprint!
tweet_listing_pd.info()

In [ ]:
# TM - top 1000 key words from Influencer tweets#SS -to 

Top_1000 = pd.Series(' '.join(tweet_listing_pd['Tweet']).lower().split()).value_counts()[:1000]

In [ ]:
#  TM - Saving the out put into a csv file
Top_1000.to_csv("RawData/Top_1000_keywords.csv", index=True, header=True)

In [ ]:
# TM -  Write to CSV for analysis
tweet_listing_pd.to_csv("RawData/TweetListings.csv", index=False, header=True)

In [ ]:
# SS - String type casting for Tweets column

tweet_listing_pd.Tweet = tweet_listing_pd.Tweet.map(''.join).str.strip()

In [ ]:
# SS - define target tags for social and entertainment

social_target_tags = ["#FamiliesBelongTogetherMarch","#gun","gun","shooting","gun-control","election","#metoo","metoo","FamiliesBelongTogetherMarch","PrideMonth","#PrideMonth","FamiliesBelongTogether","ChildreninCages","UniteTheFamilies","WeCare"]

entertainment_target_tags = ["#SocialMediaDay","SocialMediaDay","WorldCup","#WorldCup","#fifa","fifa", "#worldcup2018russia","#PostASongLyricYouLove"]


In [ ]:
# SS - function to match tweet with tags

def matcher_tweet(x):
    for i in social_target_tags:
        if i.lower() in x.lower():
            return("Social")
        
    for j in entertainment_target_tags:
        if j.lower() in x.lower():
            return("Entertainment")

In [ ]:
# SS - Add new column to tweet_listing_pd to capture Category - Social vs Entertainment

tweet_listing_pd['Tweet_Category'] = tweet_listing_pd['Tweet'].apply(matcher_tweet)

In [ ]:
# SS - Analysis of tweet_listing_pd
tweet_listing_pd_sorted = tweet_listing_pd.sort_values("Retweet_Count",ascending=False)

tweet_listing_pd_sorted.head(10)

In [ ]:
# SS - Genre vs Catgegory
tweet_listing_pd_sorted_Summary = tweet_listing_pd_sorted.groupby(["Tweet_Category","Genre"],as_index=False).count()
tweet_listing_DF = tweet_listing_pd_sorted_Summary[["Tweet_Category","Genre","Tweet"]]
tweet_listing_DF

In [ ]:
# SS  - Plot Tweet Category vs Number of Tweets

import seaborn as sns
sns.set()

sns.factorplot(x='Tweet_Category', y='Tweet', hue='Genre', data=tweet_listing_DF, kind='bar')

plt.show()

In [ ]:
#  TM - Remove the records of people who have not tweeted at all
Data_Influencers_DF = Data_Influencers_DF[Data_Influencers_DF['Tweets']>0]

In [ ]:
#  TM -  Remove missing values.
Data_Influencers_DF.dropna()

Data_Influencers_DF.to_csv('OutPut/Influencer_Analysis.csv')

In [ ]:
#  TM - Print the DF
Data_Influencers_DF

In [ ]:
#   TM - Grouped on Genre 
#   TM - Data_Influencers_DF.is_copy = False. The .copy() helps to get rid of warning.

Data_Influencers_DF['Average Tweets'] = Data_Influencers_DF['Tweets'].copy()
Data_Influencers_DF['Count'] = 0


AggregatedGenre = Data_Influencers_DF.groupby(["Genre"]).agg({'Genre': 'min', 'Tweets': 'sum', 'Followers': 'sum','Average Tweets':'mean','Age Of Account':'mean','Count':'size'})

AggregatedGenre.sort_values(['Followers'], ascending =False)


In [ ]:
#   TM - Sorting on Average Tweets and grouped on Genere
AggregatedGenre.sort_values(['Average Tweets'],ascending=False)

In [ ]:
 ## TM - whole cell
# Labels for the sections of our pie chart
labels = AggregatedGenre['Genre']

# The values of each section of the pie chart
# sizes = Data_Influencers_DF.groupby(["Genere"],as_index=False).sum()["fare"].tolist()
sizes = AggregatedGenre['Followers']

# The colors of each section of the pie chart
colors = ["gold", "lightskyblue", "lightcoral",'red','pink','darkblue','purple']

# Tells matplotlib to seperate the "Python" section from the others
explode = (0,0,0,0,0,0,0.1)

# Creates the pie chart based upon the values above
# Automatically finds the percentages of each part of the pie chart
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True, startangle=120)

# Tells matplotlib that we want a pie chart with equal axes
plt.axis("off")

  
plt.title("% Followers per Genres", fontsize=16)

plt.savefig('Images/% Followers per Genres.png')
# Prints our pie chart to the screen
plt.show()

In [ ]:
 ## TM - whole cell
# Labels for the sections of our pie chart
labels = AggregatedGenre['Genre']

# The values of each section of the pie chart
# sizes = Data_Influencers_DF.groupby(["Genere"],as_index=False).sum()["fare"].tolist()
sizes = AggregatedGenre['Tweets']

# The colors of each section of the pie chart
colors = ["gold", "lightskyblue", "lightcoral",'red','pink','darkblue','purple']

# Tells matplotlib to seperate the "Python" section from the others
explode = (0,0,0,0,0,0,0.1)

# Creates the pie chart based upon the values above
# Automatically finds the percentages of each part of the pie chart
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct="%1.1f%%", shadow=True, startangle=120)

# Tells matplotlib that we want a pie chart with equal axes
plt.axis("off")

  
plt.title("% Tweets per Genres", fontsize=16)

plt.savefig('Images/% Tweets per Genres.png')
# Prints our pie chart to the screen
plt.show()

In [ ]:
# -TM - People who are top ten Influencer who tweet more
top_ten_twitters = Data_Influencers_DF.sort_values(['Tweets'],ascending=False).head(10)
top_ten_twitters

In [ ]:
# AggregatedGenre

In [ ]:
#  -TM - Top ten Influencer who who have more Followers
top_ten_influencer = Data_Influencers_DF.sort_values(['Followers'],ascending=False).head(10)
top_ten_influencer

In [ ]:
#--RR Still Working test cell
# to find the handle of followers of the top most 
# active Influencer so that we can send the tweet through a bot.
# top_ten_twitters.head(1)['Twitter_Handle'].map(lambda x: x.lstrip('@'))
# import time
# ids = []
# for page in tweepy.Cursor(api.followers_ids, top_ten_twitters.head(1)['Twitter_Handle'].map(lambda x: x.lstrip('@'))).pages():
#     ids.extend(page)
#     time.sleep(60)

# 2. Sentiment Analysis of the Influencers: 

In [ ]:
# # -TM - Extract the first 1000 tweets of the Influencers
print("-----------Start extraction of the tweets posted by the Influencers!!!-----------")
Influencers = top_ten_influencer['Twitter_Handle']
Sentiment_array = []

for user in Influencers:
    # Set the tweet count to 100
    tweet_count = 1000
    print("Extracting tweets from %s"%user)
    
    # Extract tweets up to 5 pages
    for x in range(10):
        influencer_tweets = api.user_timeline(user,page = x)
# influencer_tweets        
        
        # For each tweet in a bunch of public tweets
        for tweet in influencer_tweets:
            
            #Calculate the compound, positive, negative and neutral values of each tweet
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]
            
            age_of_tweet = (datetime.now(timezone.utc) - datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S %z %Y')).days
            tweetAge = 0 
            if age_of_tweet < 180:
                tweetAge = 1
            elif age_of_tweet < 365:   
                tweetAge = 2
            else:
                tweetAge = 3
            
 
            
            # Save the Tweets in an array as a dictionery item 
            Sentiment_array.append({"Influencers" : user,
                                    "Tweet Text" : tweet["text"],
                                    "Compound" : compound,
                                    "Positive" : pos,
                                    "Negative" : neg,
                                    "Neutral" : neu,
                                    "Date" : datetime.strptime(tweet["created_at"],'%a %b %d %H:%M:%S %z %Y').strftime('%m/%d/%Y'),
                                    "AgeOfTweet":age_of_tweet,
                                    "TweetAgeType":tweetAge,
                                    "Tweets Ago": tweet_count
                                   })
            
            #Decrease count of tweet by 1 in the reverse order
            tweet_count -= 1

print("-----------End of Extraction of Tweets !!!-----------")

In [ ]:
# # RR Sentiment_array
# Sentiment_array['TweetAge'] = 3

# Sentiment_array.loc[Sentiment_array['AgeOfTweet'] < 180, 'TweetAge'] = 1
# Sentiment_array.loc[Sentiment_array['AgeOfTweet'] > 180 & Sentiment_array['AgeOfTweet'] < 365, 'TweetAge'] = 2
# Sentiment_array

In [ ]:
# -TM - whole Cell
# Create dataframe from the Dictionery item of the Sentiment Array
Sentiment_DF = pd.DataFrame.from_dict(Sentiment_array)

# -TM -Remove the '@' from the 'influence' column in the data frame
Sentiment_DF["Influencers"] = Sentiment_DF["Influencers"].map(lambda x: x.lstrip('@'))

# -TM - Re_arrang the columns and save into a CSV file
Sentiment_DF = Sentiment_DF[["Influencers", "Date", "Tweet Text"
                             , "Compound", "Positive", "Negative"
                             , "Neutral", "Tweets Ago","AgeOfTweet","TweetAgeType"
                            ]]

# -TM -Store output in a .CSV File
Sentiment_DF.to_csv("OutPut/influencer_Sentiment_tweets_Analysis.csv")

Sentiment_DF

In [ ]:
# -TM -Get the unique list of the influencers
Influencers_array = Sentiment_DF["Influencers"].unique()
Influencers_array

In [ ]:
# -TM -Calculate the mean for each Influencers & store into a dataframe
Influencers_Comp_Mean = Sentiment_DF.groupby("Influencers").mean()["Compound"].to_frame()

#-TM -Reset the index 
Influencers_Comp_Mean.reset_index(inplace=True)

Influencers_Comp_Mean

In [ ]:
# -TM - Whole cell
# Set the values for x_axis & y_axis
x_axis = Influencers_Comp_Mean.index.values
y_axis = Influencers_Comp_Mean["Compound"]
X_Label = Influencers

# Intialize the plots. 
fig,ax = plt.subplots()#  function that returns a tuple containing a figure and axes object(s)

#Set the plot and assign the values like colors etc
bars = ax.bar(x_axis,y_axis
              , align = "edge"
              , width = 1
              , linewidth = 1
              , edgecolor = 'black'
              , color = ["yellow","lime","red","orange","pink"]
             )

# Set the tick(s) of the bar graph
tick_locations = [value + 0.5 for value in range(len(x_axis))]
plt.xticks(tick_locations,X_Label,rotation='vertical')

# If value is positive then put True in the Summary else place False
Influencers_Comp_Mean["Positive"] = Influencers_Comp_Mean["Compound"] > 0

# Assign the height based on positive value after allocating True / false value
height = Influencers_Comp_Mean.Positive.map({True: 0.03 , False: -0.03})

# # Set the value on labels on the bars
for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + height[bars.index(bar)]
            , round(Influencers_Comp_Mean["Compound"][bars.index(bar)],3)
            , ha = 'center'
            , va = 'bottom'
            )

# Set the x_axis limits
ax.set_xlim(0, len(x_axis))

# Dynamically set the y_axis limits by finding the max & min value of y-axis
ax.set_ylim(min(y_axis)-0.1, max(y_axis) + 0.1)

# Set a horizontal line at y = 0
plt.hlines(0,0,len(x_axis))

# Title of the graph
ax.set_title("Sentiments on Twitter of Influencers (%s)" % (time.strftime("%x")), fontsize=16)

# Setting the y_axis label
ax.set_ylabel("Polarity on Twitter ", fontsize=14)

# # Setting the x_axis label
ax.set_xlabel("The Influencers", fontsize=14)
  
# Saving the graph
plt.savefig("Images/The Influencer Twitter Sentiment.png",bbox_inches='tight')
plt.show()

In [ ]:
# IB People who are top 5 Influencer, with the greatest number of followers
top_5_influencer = Data_Influencers_DF.sort_values(['Followers'],ascending=False).head(5)
top_5_influencer

In [ ]:
#IB Working Module --  Extract the first 100 tweets of the top ten Influencers

# Extract the first 100 tweets of the Influencers
print("-----------Start extraction of the tweets posted by the Influencers!!!-----------")
Influencers = []
Influencers = top_5_influencer["Twitter_Handle"]
Influencers

# IB Influencer's overall sentiments
Influencers_overall_results = []

Sentiment_array_infl = []

# IB loop through each top ten influencer
for user in Influencers:
    
   # IB Variables for holding compound sentiments
    compound_list = []
    
    # IB Variable for max_id
    oldest_tweet = None
    
    # Set the tweet count to 100
    tweet_count = 0
    print("Extracting tweets from %s"%user)
    
    # Extract tweets up to 5 pages
    for x in range(5):
        ### IB - it is working - print(f"For page number: '{x}' ")
        influencer_tweets = api.user_timeline(user,page = x,  max_id = oldest_tweet )       
        
        # For each tweet in a bunch of public tweets
        for tweet in influencer_tweets:
            
            #Calculate the compound, positive, negative and neutral values of each tweet
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]
            
            # Save the Tweets in an array as a dictionery item 
            Sentiment_array_infl.append({"Influencers" : user,
                                    "Tweet Text" : tweet["text"],
                                    "Compound" : compound,
                                    "Positive" : pos,
                                    "Negative" : neg,
                                    "Neutral" : neu,
                                    "Date" : tweet["created_at"],
                                    "Tweets Ago" : tweet_count
                                   })
            
            # IB Collect compund sentiments for each Infuencer
            compound_list.append(compound)
            
            #Decrease count of tweet by 1 in the reverse order
            tweet_count += 1
            
        # IB get Tweet ID, subtract 1,assign to oldest_tweet
        oldest_tweet = tweet["id"] - 1
        
    # IB List for dictionary of results for _overall_ sentiments for each user
    Influencers_overall_results.append({"Influencer": user, "Overall Sentiment": np.mean(compound_list)})

print("-----------End of Extraction of Tweets !!!-----------")
###Influencers_overall_results

In [ ]:
# IB Prepare Results Dataframe for the scatter plot
Influencers_overall_results_df = pd.DataFrame.from_dict(Sentiment_array_infl)
Influencers_overall_results_df.head(10)

In [ ]:
# IB Creating Sentiment Analysis for Top Ten Influences (07/02/2018) Scatter Plot

# Separate media_user_results_df Dataset into 5 parts/ datasets by "Media Source"
inf1_tweet_data_df = Influencers_overall_results_df[Influencers_overall_results_df['Influencers'] == "@katyperry"] 
inf2_tweet_data_df = Influencers_overall_results_df[Influencers_overall_results_df['Influencers'] == "@BarackObama"] 
# For each of media sources following the first one in the media dataset reset index to 0-99
# in order to plot in the common area
inf2_tweet_data_df = inf2_tweet_data_df.reset_index(drop=True)
inf3_tweet_data_df = Influencers_overall_results_df[Influencers_overall_results_df['Influencers'] == "@taylorswift13"] 
inf3_tweet_data_df = inf3_tweet_data_df.reset_index(drop=True)
inf4_tweet_data_df = Influencers_overall_results_df[Influencers_overall_results_df['Influencers'] == "@ladygaga"] 
inf4_tweet_data_df = inf4_tweet_data_df.reset_index(drop=True)
inf5_tweet_data_df = Influencers_overall_results_df[Influencers_overall_results_df['Influencers'] == "@TheEllenShow"] 
inf5_tweet_data_df = inf5_tweet_data_df.reset_index(drop=True)

# Get the values for each part of the scatter plot by "Influencers"
# INF1
# Get values for X axis
inf1_tweet_ago_data = inf1_tweet_data_df.index
# Get values for Y axis
inf1_tweet_polarity_data = inf1_tweet_data_df["Compound"] 
####################################################################
# INF2
# Get values for X axis
inf2_tweet_ago_data = inf2_tweet_data_df.index 
# Get values for Y axis
inf2_tweet_polarity_data = inf2_tweet_data_df["Compound"] 
####################################################################
# INF3
# Get values for X axis
inf3_tweet_ago_data = inf3_tweet_data_df.index
# Get values for Y axis
inf3_tweet_polarity_data = inf3_tweet_data_df["Compound"] 
####################################################################
# INF4
# Get values for X axis
inf4_tweet_ago_data = inf4_tweet_data_df.index
# Get values for Y axis
inf4_tweet_polarity_data = inf4_tweet_data_df["Compound"] 
####################################################################
# INF5
# Get values for X axis
inf5_tweet_ago_data = inf5_tweet_data_df.index
# Get values for Y axis
inf5_tweet_polarity_data = inf5_tweet_data_df["Compound"] 
####################################################################

# Organize the layout for the scatter plot
plt.title("Sentimental Analysis of Top 5 Influencers 100 Tweets(07/03/2018)", Fontsize= 10 )
# Labels for the scatter plot circles for each Influencer
labels = ["KatyPerry" "BarackObama" "TaylorSwift" "LadyGaga" "TheEllenShow"]
# Colors for the scatter plot circles for each Influencer
colors = ["lightskyblue","green","red", "blue", "gold"]

# Define X and Y of the scatter plot
plt.xlabel("Tweets Ago")
plt.ylabel("Tweet Polarity")

# Define X and Y limits
plt.xlim(100, 0)
plt.ylim( -1.0, 1.0 )

# Writing data to the same Scatter Plot 5 times, for each Influencer 
#           ("KatyPerry" "BarackObama" "TaylorSwift" "LadyGaga" "TheEllenShow")
plt.scatter( inf1_tweet_ago_data, inf1_tweet_polarity_data, c="lightskyblue", edgecolor="black", 
            linewidths=1, alpha =0.75, marker='o', label="KatyPerry")
plt.scatter( inf2_tweet_ago_data, inf2_tweet_polarity_data, c="green", edgecolor="black", \
            linewidths=1, alpha=0.75, marker='o', label="BarackObama")
plt.scatter( inf3_tweet_ago_data, inf3_tweet_polarity_data, c="red", edgecolor="black", 
            linewidths=1, alpha=0.75, marker='o', label="TaylorSwift")
plt.scatter( inf4_tweet_ago_data, inf4_tweet_polarity_data, c="blue", edgecolor="black", 
            linewidths=1, alpha=0.75, marker='o', label="LadyGaga")
plt.scatter( inf5_tweet_ago_data, inf5_tweet_polarity_data, c="gold", edgecolor="black", 
            linewidths=1, alpha=0.75, marker='o', label="TheEllenShow")
 
# Create the legend of the Scatter plot
legend = plt.legend(fontsize="small", loc="center left", title="Top 5 Influencers", bbox_to_anchor=(1, 0.5))
plt.tight_layout()

plt.grid(True)

# Save the figure with the Scatter Plot
plt.savefig("OutPut/InfluencersSentimentsTweeterPolarityScatterPlot.png")

# Show the Scatter Plot
plt.show()

In [ ]:
# IB Top 5 People who who have more Followers
top_5_twitters = Data_Influencers_DF.sort_values(['Tweets'],ascending=False).head(5)
top_5_twitters

In [ ]:
#IB Working Module --  Extract the first 100 tweets of the top Twitters

# # Extract the first 100 tweets of the Influencers
print("-----------Start extraction of the tweets posted by the Influencers!!!-----------")
Twitters = []
Twitters = top_5_twitters["Twitter_Handle"]
Twitters

# IB Twitters's overall sentiments
Twitters_overall_results = []

Sentiment_array_tw = []

# IB loop through each top ten twitter
for user in Twitters:
    
    ###print(f"For Twitter: '{user}' ")
    # IB Variables for holding compound sentiments
    compound_list_tw = []
    
    # IB Variable for max_id
    oldest_tweet = None
    
    # Set the tweet count to 100
    tweet_count = 0
    print("Extracting tweets from %s"%user)
    
    # Extract tweets up to 5 pages
    for x in range(5):
        
        ### IB - it is working - print(f"For page number: '{x}' ")
        twitters_tweets = api.user_timeline(user,page = x,  max_id = oldest_tweet )       
        
        # For each tweet in a bunch of public tweets
        for tweet in twitters_tweets:
            
            #Calculate the compound, positive, negative and neutral values of each tweet
            compound = analyzer.polarity_scores(tweet["text"])["compound"]
            pos = analyzer.polarity_scores(tweet["text"])["pos"]
            neu = analyzer.polarity_scores(tweet["text"])["neu"]
            neg = analyzer.polarity_scores(tweet["text"])["neg"]
            
            # Save the Tweets in an array as a dictionery item 
            Sentiment_array_tw.append({"Twitter" : user,
                                    "Tweet Text" : tweet["text"],
                                    "Compound" : compound,
                                    "Positive" : pos,
                                    "Negative" : neg,
                                    "Neutral" : neu,
                                    "Date" : tweet["created_at"],
                                    "Tweets Ago" : tweet_count
                                   })
            
            # IB Collect compund sentiments for each Infuencer
            compound_list_tw.append(compound)
            
            #Decrease count of tweet by 1 in the reverse order
            tweet_count += 1
            
        # IB get Tweet ID, subtract 1,assign to oldest_tweet
        oldest_tweet = tweet["id"] - 1
        
    # IB List for dictionary of results for _overall_ sentiments for each user
    Twitters_overall_results.append({"Twitter": user, "Overall Sentiment": np.mean(compound_list_tw)})

print("-----------End of Extraction of Tweets !!!-----------")
###Twitters_overall_results

In [ ]:
# IB Prepare Results Dataframe for the scatter plot
Twitters_overall_results_df = pd.DataFrame.from_dict(Sentiment_array_tw)
Twitters_overall_results_df.head(10) 

In [ ]:
# IB Creating Sentiment Analysis for Top Ten Influences Scatter Plot

# Separate media_user_results_df Dataset into 5 parts/ datasets by "Media Source"
tw1_tweet_data_df = Twitters_overall_results_df[Twitters_overall_results_df['Twitter'] == "@Noahpinion"] 
tw2_tweet_data_df = Twitters_overall_results_df[Twitters_overall_results_df['Twitter'] == "@BDUTT"] 
# For each of media sources following the first one in the media dataset reset index to 0-99
# in order to plot in the common area
tw2_tweet_data_df = tw2_tweet_data_df.reset_index(drop=True)
tw3_tweet_data_df = Twitters_overall_results_df[Twitters_overall_results_df['Twitter'] == "@ProfessorChic"] 
tw3_tweet_data_df = tw3_tweet_data_df.reset_index(drop=True)
tw4_tweet_data_df = Twitters_overall_results_df[Twitters_overall_results_df['Twitter'] == "@haroldpollack"] 
tw4_tweet_data_df = tw4_tweet_data_df.reset_index(drop=True)
tw5_tweet_data_df = Twitters_overall_results_df[Twitters_overall_results_df['Twitter'] == "@CoryBooker"] 
tw5_tweet_data_df = tw5_tweet_data_df.reset_index(drop=True)

# Get the values for each part of the scatter plot by "Twitter"
# TW1
# Get values for X axis
tw1_tweet_ago_data = tw1_tweet_data_df.index
# Get values for Y axis
tw1_tweet_polarity_data = tw1_tweet_data_df["Compound"] 
####################################################################
# TW2
# Get values for X axis
tw2_tweet_ago_data = tw2_tweet_data_df.index 
# Get values for Y axis
tw2_tweet_polarity_data = tw2_tweet_data_df["Compound"] 
####################################################################
# TW3
# Get values for X axis
tw3_tweet_ago_data = tw3_tweet_data_df.index
# Get values for Y axis
tw3_tweet_polarity_data = tw3_tweet_data_df["Compound"] 
####################################################################
# TW4
# Get values for X axis
tw4_tweet_ago_data = tw4_tweet_data_df.index
# Get values for Y axis
tw4_tweet_polarity_data = tw4_tweet_data_df["Compound"] 
####################################################################
# TW5
# Get values for X axis
tw5_tweet_ago_data = tw5_tweet_data_df.index
# Get values for Y axis
tw5_tweet_polarity_data = tw5_tweet_data_df["Compound"] 
###################################################################

# Organize the layout for the scatter plot
plt.title("Sentimental Analysis of Top 5 Tweeters Tweets(07/03/2018)", Fontsize= 10 )

# Labels for the scatter plot circles for each Twitter
labels = ["Noahpinion" "BDUTT" "ProfessorChic" "HaroldPollack" "CoryBooker"]
# Colors for the scatter plot circles for each Twitter
colors = ["lightskyblue","green","red", "blue", "gold"]

# Define X and Y of the scatter plot
plt.xlabel("Tweets Ago")
plt.ylabel("Tweet Polarity")

# Define X and Y limits
plt.xlim(100, 0)
plt.ylim( -1.0, 1.0 )

# Writing data to the same Scatter Plot 5 times, for each Twitter 
#           ( "Noahpinion" "BDUTT" "ProfessorChic" "HaroldPollack" "CoryBooker")
plt.scatter( tw1_tweet_ago_data, tw1_tweet_polarity_data, c="lightskyblue", edgecolor="black", 
            linewidths=1, alpha =0.75, marker='o', label="Noahpinion")
plt.scatter( tw2_tweet_ago_data, tw2_tweet_polarity_data, c="green", edgecolor="black", \
            linewidths=1, alpha=0.75, marker='o', label="BDUTT")
plt.scatter( tw3_tweet_ago_data, tw3_tweet_polarity_data, c="red", edgecolor="black", 
            linewidths=1, alpha=0.75, marker='o', label="ProfessorChic")
plt.scatter( tw4_tweet_ago_data, tw4_tweet_polarity_data, c="blue", edgecolor="black", 
            linewidths=1, alpha=0.75, marker='o', label="HaroldPollack")
plt.scatter( tw5_tweet_ago_data, tw5_tweet_polarity_data, c="gold", edgecolor="black", 
            linewidths=1, alpha=0.75, marker='o', label="CoryBooker")
 
# Create the legend of the Scatter plot
legend = plt.legend(fontsize="small", loc="center left", title="Top 5 Influencers", bbox_to_anchor=(1, 0.5))
plt.tight_layout()

plt.grid(True)

# Save the figure with the Scatter Plot
plt.savefig("OutPut/TweetersSentimentsTweeterPolarityScatterPlot.png")

# Show the Scatter Plot
plt.show()

In [ ]:
# WIP
# influencersDictionary = {}
# colors = ["yellow","lime","red","orange","pink","yellow","lime","red","orange","pink"]
# index = 0
# for person in Influencers:
#     influencersDictionary[person] = colors[index]
#     index+=1
# influencersDictionary

In [ ]:
# # Create an array of Influencers Houses with the unique function in the data frame
# Influencers_array = Sentiment_DF["Influencers"].unique()
# Influencers_array

# # #Plotting the graph for each influencer
# for influencer in Influencers_array:
# # Creating a temporary data frame to store for only one influencer at a time
#         Temp_DF = Sentiment_DF[Sentiment_DF["influencer"] == influencer]
        
#         Sentiment_DF['influencer'] = Sentiment_DF['influencer'].map(lambda x: x.lstrip('@'))
# #Temp_DF
    
#         plt.scatter(Temp_DF["Tweets Ago"],Temp_DF["Compound"]
#                  , marker = "o", linewidth = 0, alpha = 0.8, label = Influencers
#                  , facecolors = Temp_DF.influencer.map(influencersDictionary)
#                 )

# # # Set the legend 
# plt.legend(bbox_to_anchor = (1,1), title="The Influencers", loc='best')

# # # Set the labels of x_axis, y_axis & title 
# plt.xlabel("Tweets Ago", fontsize=12)
# plt.ylabel("Tweet Polarity", fontsize=12)
# plt.title("Sentiment Analysis of The Influencers Tweets (%s)" % (time.strftime("%x")), fontsize=16)

# # #Set the limite of  x_axis and y_axis
# plt.xlim(0, 101)
# plt.ylim(-1,1)

# # # Set the grid
# plt.grid(True)

# filePath = 'Images'
# if not path.exists(filePath):
#     makedirs(filePath)

# # Save the result to a .png file
# plt.savefig("Sentiment Analysis of Influencers Tweets.png",bbox_inches='tight')
# # plt.savefig("Sentiment Analysis of The influencer's Tweets.png",bbox_inches='tight')

# plt.show()

In [ ]:
# Version: 2.0
# Date: Mon July 2nd, 2018 
# Time: 10:25 PM

# Functionalities: 
# A) Data Analysis
#     1. Created dataframe.
#     2. Sorting
# B) Sentiment Analysis

# WIP (Work in progress)
# 1. Tweet bot. The git part is done. 
# Automation is pending as hosting might need neet tech which we have not covered now.

# Pending items:
# 1. Data Cleaning
# 2. Graphs and plots
# 3. Testing

In [ ]:
# Version: 3.0 - IB
# Date: Monday 7/3/2018
# Time: 3:00 PM
# Functionalities: 
# A) Overcame _TweepError - 34_ by using max_id logic while calling api.user_timeline Tweepy function
#    Sentiment Analysis for Top 5 Influencers and Top 5 Twitters function 
# B) Created the Scatter Plot for Sentimental Analysis of Top 5 Influencers last 100 tweets
# C) Created the Scatter Plot for Sentimental Analysis of Top 5 Twitters last 100 tweets
#    Scatter Plots png files are saved in OutPut directory
#
# These Scatter Plots visualize a great difference between _overal_ ratings of Influencers and People who tweet most
# 
# For Merging: Code has # IB comments, 8 cells. When "Restart and Run All" Kernel, my code is in ln[37] to ln[44]
# With amount of tweets we have it runs under 10 minutes